In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

def parser(url):
    """Get a parsed version of an URL"""
    # Get an URL
    try:
      res = requests.get(url)
    except Exception as err:
      print('There was a problem: {}'.format(err))
    
    #Parse the URL above
    soup = BeautifulSoup(res.text)
    
    return soup

In [2]:
def get_urls(url):
    """Get the URLs of all categories on Tiki homepage"""
    categ_list = []
    
    # Parse the URL
    s = parser(url)
    
    # Take url and category then add them into List
    for i in s.find_all('a', class_='MenuItem__MenuLink-tii3xq-1 efuIbv'):
        url = i.get('href')
        category = i.find('span', class_='text').text
        categ_list.append((category, url))
        
    return categ_list
        

In [3]:
def scrape(cat, url):
    """Scrape product information of all products on a page"""
    # Initialize an empty list
    results = []
    
    # Parse
    s = parser(url)
    
    # Find all DIV TAG with class =  'product item'
    product_items = s.find_all('div', class_='product-item')
    
    # If there is no item return empty list, otherwise, take each of element from list, then take specific points in this element
    if product_items == 0:
        return []
    else:
        for product_item in product_items:
            row = [product_item['data-id'],
                   product_item['data-seller-product-id'],
                   product_item['data-title'],
                   product_item['data-price'],
                   product_item.img['src'],
                   cat
                  ]
            results.append(row)
            
    return results

In [149]:
url_list = get_urls('https://tiki.vn')
url_list
test_scraper = scrape(url_list[0][0], url_list[0][1])
test_scraper
df = pd.DataFrame(data=test_scraper, columns = ['product_id', 'seller_id', 'title', 'price', 'image_url', 'category' ])
df.sample(10)

,product_id,seller_id,title,price,image_url,category
27,1457747,16276108,Thẻ Nhớ Micro SDXC SanDisk UHS-1 64GB Class 10...,187900,https://salt.tikicdn.com/cache/200x200/ts/prod...,Phụ Kiện - Thiết Bị Số
31,7030139,8095990,Tai nghe Bluetooth 4.1 tích hợp nam châm,56600,https://salt.tikicdn.com/cache/200x200/ts/prod...,Phụ Kiện - Thiết Bị Số
20,12705571,12705572,Tai Nghe Bluetooth Apple AirPods 2 True Wirele...,4852000,https://salt.tikicdn.com/cache/200x200/ts/prod...,Phụ Kiện - Thiết Bị Số
13,6215251,21183983,Tai nghe bluetooth 4.2 MagnerS6,39000,https://salt.tikicdn.com/cache/200x200/ts/prod...,Phụ Kiện - Thiết Bị Số
41,4897521,4897525,Tai Nghe Nhét Tai JBL C150SI - Hàng Chính Hãng,185000,https://salt.tikicdn.com/cache/200x200/ts/prod...,Phụ Kiện - Thiết Bị Số
18,589242,589272,Vòng Đeo Tay Thông Minh Xiaomi Mi Band 2 (Đen)...,309000,https://salt.tikicdn.com/cache/200x200/ts/prod...,Phụ Kiện - Thiết Bị Số
22,299461,299484,Chuột Không Dây Logitech M331 Silent - Hàng Ch...,297000,https://salt.tikicdn.com/cache/200x200/media/c...,Phụ Kiện - Thiết Bị Số
39,2541643,12522617,Lót chuột Bản Đồ Thế Giới cực chất (80x 30cm),64920,https://salt.tikicdn.com/cache/200x200/ts/prod...,Phụ Kiện - Thiết Bị Số
2,1097744,19882317,Loa Bluetooth JBL Pulse 3 20W - Hàng Chính Hãng,4370000,https://salt.tikicdn.com/cache/200x200/ts/prod...,Phụ Kiện - Thiết Bị Số
28,1205143,5755907,Cáp Sạc Điện Thoại Foxconn cho Iphone 5/6/7/8 ...,24999,https://salt.tikicdn.com/cache/200x200/ts/prod...,Phụ Kiện - Thiết Bị Số


In [8]:
def scrape_all():
    """Scrape all products on Tiki!"""
    print('INFO scrape_all(): Start craping')
    
    # Initialize empty 'queue' list with the value of get_urls() function
    queue = get_urls('https://tiki.vn')
    
    # Initialize empty list
    results = []
    
    # Set page = 1 as default
    page = 1
    
    # Take the last category in the list, then Pop it until there is no category 
    while len(queue) > 0:
        url = queue[len(queue)-1][1]
        cat = queue[len(queue)-1][0]
        queue.pop()
        
        print(url)
        print('Scraping', cat)
        
        # Take a list of item in 1 page
        new_rows = scrape(cat, url)
        print(new_rows)
        
        # if this current page has product, continue to take product on next page until no product
        if len(new_rows) > 0:
            results += new_rows
            #page = int(url[url.rfind('&page=')+6:]) + 1
            #url = url[:url.rfind('&page=')+6] + str(page)
            page += 1
            url = url.split('&page=')[0] + "&page=" + str(page)
            
            # The last element of this list [category, link] is always the link of current page
            queue.append([page, url])
        else:
            page = 1
          
    return results       

In [234]:
url = 'https://tiki.vn/dien-thoai-may-tinh-bang/c1789?src=c.1789.hamburger_menu_fly_out_banner&page=2'
# page = url.rfind('&page=')
# page
# int(url[url.rfind('&page=')+6:]) + 1
page = int(url[url.rfind('&page=')+6:]) + 1
# url[:page]
# url[:url.rfind('&page=')+6] + page
page

url[:url.rfind('&page=')+6] + str(page)

'https://tiki.vn/dien-thoai-may-tinh-bang/c1789?src=c.1789.hamburger_menu_fly_out_banner&page=3'

In [9]:
a = scrape_all()
df = pd.DataFrame(data=a, columns = ['product_id', 'seller_id', 'title', 'price', 'image_url', 'category' ])
df.sample(10)

INFO scrape_all(): Start craping
https://tiki.vn/voucher-dich-vu/c11312?src=c.11312.hamburger_menu_fly_out_banner
Scraping Voucher - Dịch Vụ - Thẻ Cào
[['16563156', '16563157', 'Aka House - Đại Tiệc Buffet Nướng & Lẩu Nhật Bản Hơn 90 Món Không Giới Hạn Áp Dụng Toàn Quốc', '229000', 'https://salt.tikicdn.com/cache/200x200/ts/product/eb/f9/b4/e719f60ff0b963c8ff63e0253e91ccd2.jpg', 'Voucher - Dịch Vụ - Thẻ Cào'], ['20475731', '20475732', 'Kohaku Hotpot - Buffet Lẩu Nướng Hải Sản, Bò Mỹ Cao Cấp Menu 384k', '279000', 'https://salt.tikicdn.com/cache/200x200/ts/product/28/a9/27/a502b3937dc8c56aee838080e203ae4a.jpg', 'Voucher - Dịch Vụ - Thẻ Cào'], ['20472849', '20472853', 'Kohaku Hotpot - Buffet Nướng Hải Sản, Bò Mỹ Cao Cấp Menu 318k', '229000', 'https://salt.tikicdn.com/cache/200x200/ts/product/26/69/8e/2d91cc4bdab978048b1eb204d86157e4.jpg', 'Voucher - Dịch Vụ - Thẻ Cào'], ['1918707', '1940707', 'Buffet Tối Cuối Tuần 70 món - Nhà Hàng Liberty Hotel Saigon Parkview', '283000', 'https://salt.t

[['2376893', '2376895', 'Trượt băng nghệ thuật không giới hạn tại Sân trượt băng Vincom - Áp dụng cả tuần', '119000', 'https://salt.tikicdn.com/cache/200x200/ts/product/d9/4c/88/0fa33fa3a0be033e42de80e459308f84.jpg', 2], ['19240181', '19240182', 'Marina Bay Vũng Tàu Resort & Spa 5* - Sát Biển, Phòng Deluxe Hướng Biển, Hồ Bơi Vô Cực Đẹp Xuất Sắc', '2450000', 'https://salt.tikicdn.com/cache/200x200/ts/product/41/37/5f/f5171bb40af535dc7c99e934d981f452.jpg', 2], ['14615616', '14615617', 'Lotte Cinema Solo Combo - 1 Bắp Rang Lớn Và 1 Nước Ngọt Lớn', '65000', 'https://salt.tikicdn.com/cache/200x200/ts/product/c5/9d/41/4dcf799a77eafbbf0b2c7c0ee79c4938.jpg', 2], ['1550141', '1550143', 'Tour Thái Lan: Bangkok - Pattaya - Đảo Coral 5N4Đ Khởi Hành Tháng 08 & Lễ Quốc Khánh', '5619000', 'https://salt.tikicdn.com/cache/200x200/ts/product/5e/48/44/b0023f93401d77c21ea0da68dfc7f205.jpg', 2], ['4401241', '4401243', 'Cô Quyên Xứ Quảng - Áp Dụng Toàn Menu Món Ăn & Thức Uống Gần 40 Món Hương Vị Miền Trung'

[['10572135', '10572136', 'Khóa học Yoga bầu - Thai giáo', '199000', 'https://salt.tikicdn.com/cache/200x200/ts/product/e4/01/1f/b9bab5f6b1937eeb4e52c11a5e86a569.jpg', 3], ['5061783', '5061785', 'Tour 6N5Đ Phượng Hoàng Cổ Trấn - Trương Gia Giới, Bay Thẳng Vietjet Air Từ HCM', '14575000', 'https://salt.tikicdn.com/cache/200x200/ts/product/1b/e7/04/f922c29bf3a0021a751e29579681cd39.jpg', 3], ['8113007', '8113008', 'Liệu Trình Massage Thư Giãn Giải Độc Cơ Thể Tại Les Sam Wellness & Spa', '149000', 'https://salt.tikicdn.com/cache/200x200/ts/product/e2/12/bb/57ad3aafad2e9af1e60bf18baea9a384.jpg', 3], ['6479607', '6479609', 'Gohub - Sim 4G Nhật Bản 7GB (DATA)', '408500', 'https://salt.tikicdn.com/cache/200x200/ts/product/c1/2e/d9/7397c5a17346855f73765481071e95d3.jpg', 3], ['7276139', '7276141', 'Chuẩn Massage - Gói Vip Massage Body 90 Phút', '299000', 'https://salt.tikicdn.com/cache/200x200/ts/product/af/c1/19/38951b5dbaedb27d36989d8b47eb2767.jpg', 3], ['10122318', '10122319', '[Tour VIP] Hạ 

[['19846008', '19846009', 'Ứng Dụng Học Toán Tiếng Anh Monkey Math - Gói 12 Tháng', '474050', 'https://salt.tikicdn.com/cache/200x200/ts/product/0a/64/bf/9d78fd11e984082256682447048c1772.jpg', 4], ['9795269', '9795270', 'Bevita – Liệu Trình Điều Trị Mụn Chuyên Sâu (Cam Kết 100%)', '99000', 'https://salt.tikicdn.com/cache/200x200/ts/product/d5/0f/d0/5ddb27f923c10df557648b55fa996602.jpg', 4], ['6939321', '6939323', 'Miễn Tip - (90’) Massage Body + Foot + Chạy Vitamin C + Đắp Mặt Nạ', '99000', 'https://salt.tikicdn.com/cache/200x200/ts/product/72/8f/2f/441de0d47579710f1d808dc150bc9c14.jpg', 4], ['4562367', '4584879', 'WikiLady - Combo 3 Khóa Học Làm Thạch Rau Câu Từ Cơ Bản Đến Nâng Cao Thiên Nga - Công - Hạc', '1500000', 'https://salt.tikicdn.com/cache/200x200/ts/product/d6/9a/cc/b2aa59d10623031fe2a7af3ede5a59d1.jpg', 4], ['4328405', '4328407', 'Voucher Chụp Ảnh Bé Yêu / Bé Cùng Gia Đình Tại Studio Doremon (Tặng Ảnh 15 x 21 cm)', '239000', 'https://salt.tikicdn.com/cache/200x200/ts/produc

[['7358865', '7358867', 'Tẩy Trắng Răng Bằng Đèn Plasma tại Hệ thống nha khoa Hợp Nhất', '400000', 'https://salt.tikicdn.com/cache/200x200/ts/product/27/ef/96/aa57fd790969a848df2ed4c38b9b8b4c.jpg', 5], ['1822197', '1889695', 'Sim 4G Hàn Quốc', '335000', 'https://salt.tikicdn.com/cache/200x200/ts/product/0e/d5/91/2e7d6829d87ea63656920e1d24f8cce1.jpg', 5], ['7322047', '7322049', 'Phiếu Quà Tặng Tiki 400.000đ', '400000', 'https://salt.tikicdn.com/cache/200x200/ts/product/c8/34/5a/294d9f907abfb8be36039a79160f38f9.jpg', 5], ['4069197', '4069199', 'Tour Sapa - Gồm Cáp Treo Fansipan 3Đ2N, KS 3*, Xe Giường Nằm, Khởi Hành Hàng Ngày Từ HN', '2320000', 'https://salt.tikicdn.com/cache/200x200/ts/product/df/33/d3/24b5a340ea021694bdfb0716b4b3df36.jpg', 5], ['19842184', '19897993', 'Ứng Dụng Học Ngôn Ngữ Monkey Junior - Gói 48 Tháng', '1113000', 'https://salt.tikicdn.com/cache/200x200/ts/product/2d/03/d1/3823be91ee765d32f4ff2daf0a39e551.jpg', 5], ['10906367', '10906368', 'Procare24H - Dịch Vụ Cài Win

...

[['21787118', '22011513', 'Tick Edu - Thai Giáo - Phát Triển Trí Tuệ Và Cảm Xúc Cho Con Từ Trong Bụng Mẹ', '699000', 'https://salt.tikicdn.com/cache/200x200/ts/product/45/3d/0d/6d7e3a697e90e4253905d70bcba46b82.jpg', 25], ['21761403', '21941813', 'Tick Edu - Email Marketing - Tăng Sale Và Thương Hệu Với Chi Phí 0 Đồng', '799000', 'https://salt.tikicdn.com/cache/200x200/ts/product/e6/28/02/7521c78044335cd5757b55d4e5d48ef9.jpg', 25], ['21879147', '22022788', 'Tick Edu - Dạy Con Tự Vệ', '599000', 'https://salt.tikicdn.com/cache/200x200/ts/product/68/17/56/b099877c95f9ef5ef4bef617015e0a79.jpg', 25], ['21879159', '22022804', 'Tick Edu - Học Cách Sử Dụng Git & Github Cho Lập Trình Và Thiết Kế Website', '699000', 'https://salt.tikicdn.com/cache/200x200/ts/product/18/14/90/7785b207c47a21f25de8de374af3e3b5.jpg', 25], ['21776144', '22012728', 'Tick Edu - Khai Sáng Tiềm Năng Thiên Bẩm Cho Con', '699000', 'https://salt.tikicdn.com/cache/200x200/ts/product/75/80/2c/3decd3483b1279a2caf454f1b9e2565b.j

[['10921722', '10921723', 'Dịch Vụ Thay Nút Home iPad Mini 1 Tại Procare24H', '315000', 'https://salt.tikicdn.com/cache/200x200/ts/product/34/ba/43/d1b0065862b5492076e29ed050bb9b8c.jpg', 26], ['19749515', '19749516', 'Thiết kế ảnh trên điện thoại từ A-Z', '149000', 'https://salt.tikicdn.com/cache/200x200/ts/product/34/62/61/187392288e93c86906bdbc9e1885c64c.jpg', 26], ['14650512', '14650514', 'Dịch vụ thay vỏ điện thoại iPhone 7, 7 Plus, 8, 8 Plus', '700000', 'https://salt.tikicdn.com/cache/200x200/ts/product/c5/bd/70/0b64c56265b4092a1f2959ac11afd3db.jpg', 26], ['6522959', '6524029', 'VinOasis Phú Quốc 4* - Giá Mùa Cao Điểm', '1300000', 'https://salt.tikicdn.com/cache/200x200/ts/product/6d/56/2a/60dc14c0c7b35a58e4afe02f13028981.jpg', 26], ['16854508', '16920152', 'Dịch Vụ Thay Pin Macbook Pro 17 inch A1297 (2011) Mã Pin A1383 Tại Zfix.vn', '1300000', 'https://salt.tikicdn.com/cache/200x200/ts/product/73/6d/5f/1c86424a115a55bba53e31a98c79bb98.jpg', 26], ['9667054', '9667056', 'Rolex Hote

[['11861264', '11861265', 'Tour Phú Quốc 3N2Đ Gồm Vé Máy Bay, Đông Nam Đảo - Sunset Sanato - Bãi Sao, Khởi Hành Thứ 6 Hàng Tuần', '4350000', 'https://salt.tikicdn.com/cache/200x200/ts/product/5e/3d/46/0bc0f915cb067c33f2c714f03a153973.jpg', 27], ['11875632', '11875633', 'Tour 4N4Đ HCM - Hà Nội - Hà Giang - Lũng Cú - Sapa, Gồm Vé Máy Bay, Khởi Hành Thứ 4 Hàng Tuần', '8250000', 'https://salt.tikicdn.com/cache/200x200/ts/product/f1/76/f8/06bc5794f458d3e255b4b84958356d1f.jpg', 27], ['13446529', '13446530', 'Tour Đảo Bình Hưng - Ninh Chữ - Vĩnh Hy 2N2Đ, Khởi Hành Thứ 6 Hàng Tuần & Dịp Lễ', '1249000', 'https://salt.tikicdn.com/cache/200x200/ts/product/10/7a/2e/9b2c5e2a8a135ccc97ed2d50ca10aaef.jpg', 27], ['12081389', '12081390', 'Tour 2N2Đ Cà Mau - Bạc Liêu - Sóc Trăng, KS 5 Sao, Khởi Hành Thứ 6 Hàng Tuần & Dịp Lễ', '1540000', 'https://salt.tikicdn.com/cache/200x200/ts/product/16/a9/b0/f1096346af96337aef900cb88ba4b3c3.jpg', 27], ['11997484', '11997486', 'Tour 2N1Đ Côn Đảo - Gồm Vé Máy Bay, Khở

[['21776144', '22012728', 'Tick Edu - Khai Sáng Tiềm Năng Thiên Bẩm Cho Con', '699000', 'https://salt.tikicdn.com/cache/200x200/ts/product/75/80/2c/3decd3483b1279a2caf454f1b9e2565b.jpg', 28], ['21776153', '22012732', 'Tick Edu -Yêu Con Đúng Cách', '399000', 'https://salt.tikicdn.com/cache/200x200/ts/product/be/ee/3c/232fd41e9d9be38cb005b2f2819a4449.jpg', 28], ['21776192', '22012739', 'Tick Edu - Ăn Dặm Đúng Cách Cho Trẻ Từ 6-12 Tháng', '699000', 'https://salt.tikicdn.com/cache/200x200/ts/product/32/c7/ca/955c25ea16d5c376cb4db75414938093.jpg', 28], ['21776169', '22012735', 'Tick Edu -Đột Phá Tư Duy - Thay Đổi Cuộc Đời', '799000', 'https://salt.tikicdn.com/cache/200x200/ts/product/4a/96/5d/0e2edc8a9fcc7b8ff40ba6dcd27cb3df.jpg', 28], ['21879154', '22022800', 'Tick Edu - Đầu Tư Chứng Khoán Hiệu Quả', '599000', 'https://salt.tikicdn.com/cache/200x200/ts/product/8b/cc/af/62e0f43f909652f41e85ffdc0cec23af.jpg', 28], ['21776171', '22012719', 'Tick Edu - Luyện Phản Xạ Giao Tiếp Siêu Hiệu Quả', '

KeyboardInterrupt: 

In [13]:
df = pd.DataFrame(data=a, columns = ['product_id', 'seller_id', 'title', 'price', 'image_url', 'category' ])
df.sample(10)

,product_id,seller_id,title,price,image_url,category
118818,7156483,7156485,Ốp Lưng Cường Lực Trong Suốt Cho IPhone XS Max...,280000,https://salt.tikicdn.com/cache/200x200/ts/prod...,39
113737,23359014,23359015,Vỏ Case máy tính ASUS TUF Gaming GT501 Mid-Tow...,3990000,https://salt.tikicdn.com/cache/200x200/ts/prod...,142
56616,15344275,16177493,Kem Dưỡng Và Làm Mềm Da ISIS Pharma SECALIA Ba...,550000,https://salt.tikicdn.com/cache/200x200/ts/prod...,188
122476,14865874,14865875,Ốp lưng dành cho Vsmart Joy 1 mẫu Anime em mèo,86000,https://salt.tikicdn.com/cache/200x200/ts/prod...,116
49177,13380647,13380648,Xịt chống nắng Hàn Quốc pha lê tuyết mát lạnh ...,141500,https://salt.tikicdn.com/cache/200x200/ts/prod...,33
20331,11464760,11464761,"Real Lives, Real Listening - Intermediate",658000,https://salt.tikicdn.com/cache/200x200/ts/prod...,118
112066,2069749,2098653,Sạc Dành Cho Laptop HP 19.5V - 3.33A Đầu Hai K...,200000,https://salt.tikicdn.com/cache/200x200/ts/prod...,107
53886,5465125,7713553,Mặt nạ giấy thảo dược lô hội Welcos,12000,https://salt.tikicdn.com/cache/200x200/ts/prod...,131
14793,13194979,13194980,Bộ Dụng Cụ Mở Khóa Chuyên Nghiệp (38 Cái),346000,https://salt.tikicdn.com/cache/200x200/ts/prod...,3
15043,8912358,8912384,Bộ Thổi Bong Bóng Xà Phòng Hàng Nhập Khẩu Đức ...,73000,https://salt.tikicdn.com/cache/200x200/ts/prod...,8
